In [3]:
%pip install -q -U matplotlib numpy pandas scikit-learn seaborn plotly nbformat scipy

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.26.4 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
!

In [1]:
import os

import numpy as np
import pandas as pd
import plotly.express as px

### 1. Use Vectorized Operations Over Loops

- **Removing missing structural data** and **distinguishing between general and refined sets** can be significantly optimized by avoiding explicit loops. Pandas and NumPy support vectorized operations that are more efficient for these tasks.

### 2. Efficient File Existence Check

- Instead of constructing the file path and checking if it is a directory for each PDB ID, it's more efficient to read the directory contents once and then check membership. This reduces the number of file system operations, which are relatively slow.

### 3. Handling File Paths

- Use `os.path.join` for constructing file paths to ensure the code is cross-platform (Windows, Linux, macOS).

### 4. Improved NaN Checking

- Directly use `data['-log(Kd/Ki)'].isnull().any()` in the if condition without comparing it to `False`.

### 5. Plotting Enhancements

- The plotting section can be improved for readability and customization. For instance, `sns.distplot` is deprecated in newer versions of seaborn; using `sns.histplot` with appropriate arguments would be better.
- Adjust the plot's appearance for clarity, such as by adding labels or adjusting the bin size for histograms.

### 6. Error Handling

- Add error handling to manage scenarios where input files or directories are not found, or the output file cannot be written.

### 7. Code Comments and Docstrings

- Improve code comments and docstrings for better readability and maintainability. Ensure the docstring accurately describes all parameters and the function's behavior.

### 8. Improved plotting using plotly

- Use Plotly for interactive and more visually appealing plots. Plotly provides a wide range of customization options and interactivity features that can enhance the visualization of the data.


In [2]:
features_file_path = "HAC-Net/HACNet/element_features.xml"

In [3]:
def create_cleaned_dataset(
    PDBbind_dataset_path,
    general_set_PDBs_path,
    refined_set_PDBs_path,
    output_name,
    plot=False,
    verbose=True,
):
    """
    Produces a csv file containing PDB id, binding affinity, and set (general/refined)

    Inputs:
    1) PDBbind_dataset_path: path to PDBbind dataset; dataset is included in github repository as 'PDBbind_2020_data.csv'
    2) general_set_PDBs_path: path to PDBbind general set excluding refined set PDBs
    3) refined_set_PDBs_path: path to PDBbind refined set PDBs
    4) output_name: name for the output csv file. Must end in .csv
    5) plot = True will generate a plot of density as a function of binding affinity for general
    and refined sets

    Output:
    1) A cleaned csv containing PDB id, binding affinity, and set (general/refined):
    'output_name.csv'
    """
    # Load dataset
    data = pd.read_csv(PDBbind_dataset_path)
    if verbose:
        print(f"Loaded dataset from {PDBbind_dataset_path} with {len(data)} entries.")

    # Check for NaNs in affinity data
    if data["-log(Kd/Ki)"].isnull().any():
        if verbose:
            print("There are NaNs present in affinity data!")

    # Efficiently check for missing structural data
    general_pdb_set = set(os.listdir(general_set_PDBs_path))
    refined_pdb_set = set(os.listdir(refined_set_PDBs_path))
    if verbose:
        print(
            f"Found {len(general_pdb_set)} PDBs in general set, {len(refined_pdb_set)} in refined set."
        )

    data["set"] = data["PDB ID"].apply(
        lambda x: (
            "general"
            if x in general_pdb_set
            else ("refined" if x in refined_pdb_set else np.nan)
        )
    )
    initial_length = len(data)
    data.dropna(subset=["set"], inplace=True)
    if verbose:
        print(f"Removed {initial_length - len(data)} entries without structural data.")

    # Write out csv of cleaned dataset
    data[["PDB ID", "-log(Kd/Ki)", "set"]].to_csv(output_name, index=False)
    if verbose:
        print(f"Cleaned dataset written to {output_name} with {len(data)} entries.")

    # Plot if required
    if plot:
        fig = px.histogram(
            data,
            x="-log(Kd/Ki)",
            color="set",
            barmode="overlay",
            histnorm="density",
            title="Density of Binding Affinity by Set",
        )
        fig.update_layout(xaxis_title="-log(Kd/Ki)", yaxis_title="Density")
        fig.show()
        if verbose:
            print(
                "Interactive plot generated showing the density of binding affinity by set."
            )

In [4]:
# create cleaned dataset
create_cleaned_dataset(
    PDBbind_dataset_path="HAC-Net/input_files/PDBbind_2020_data.csv",
    general_set_PDBs_path="PDBbind/general-set/",
    refined_set_PDBs_path="PDBbind/refined-set/",
    output_name="PDBbind/PDBbind_cleaned_dataset.csv",
    plot=True,
)

FileNotFoundError: [Errno 2] No such file or directory: 'HAC-Net/input_files/PDBbind_2020_data.csv'

## 2) Add hydrogens to pocket PDB files and convert to mol2 file type, remove TIP3P atoms from mol2 files


In [5]:
# import packages
import pickle

import openbabel.pybel
from biopandas.mol2 import PandasMol2
from biopandas.pdb import PandasPdb
from rich.jupyter import print

In [6]:
refined_dataset_path = "PDBbind/refined-set/"

In [7]:
# Get all the folders in the refined dataset
refined_folders = [
    f
    for f in os.listdir(refined_dataset_path)
    if os.path.isdir(os.path.join(refined_dataset_path, f))
]
print(f"Found {len(refined_folders)} folders in the refined dataset.")


sample_folder = refined_folders[100]
print(f"Sample folder: {sample_folder}")

sample_files = os.listdir(os.path.join(refined_dataset_path, sample_folder))
print(f"Sample files: {sample_files}")

FileNotFoundError: [Errno 2] No such file or directory: 'PDBbind/refined-set/'

In [8]:
sample_protein_pdb = "1dhj_protein.pdb"
sample_ligand_mol2 = "1dhj_ligand.mol2"
sample_data_path = "sample_data"

Changes made -

- Added error handling for file input operations using try and except blocks. If the protein PDB or ligand MOL2 file cannot be read, an error message is printed, and the function returns None.
- Added a distance_threshold parameter with a default value of 8.0 Angstroms. This allows customization of the distance threshold used for determining the pocket residues and heteroatoms.
- Added an output_path parameter with a default value of None. If provided, the pocket PDB file will be saved to the specified path. If not provided, the default output file name "pocket.pdb" will be used.
- Added error handling for saving the pocket PDB file. If the file cannot be saved, an error message is printed.
- The function now returns the PandasPdb object (pred_pocket) instead of saving the file directly. This allows the caller to decide whether to save the file or use the object for further processing.
- Improved code readability by adding comments and spacing.
- Imported the cKDTree class from scipy.spatial for efficient nearest neighbor searches.
- Created k-d trees for protein atoms and heteroatoms using their coordinates.
- Used query_ball_point method of the k-d trees to find nearby protein atoms and heteroatoms within the specified distance threshold of ligand atoms.
- Simplified the extraction of unique pocket residues and heteroatoms using numpy functions and pandas methods.
- Filtered protein atoms and heteroatoms based on the pocket residues and heteroatoms using pandas methods
- Added a docstring to the extract_pocket function, describing its purpose, parameters, and return value.
- Added an include_heteroatoms parameter (default: False) to allow the user to choose whether to include all heteroatoms in the pocket or only water molecules. If set to True, all heteroatoms within the distance threshold will be included in the pocket.
- Updated the filtering of protein heteroatoms based on the include_heteroatoms parameter. If include_heteroatoms is True, all nearby heteroatoms are included. - If False (default), only water molecules (residue name "HOH") are included.
  Added comments throughout the code to explain the purpose and functionality of each section.


In [9]:
import os

from rich.jupyter import print
from scipy.spatial import cKDTree


def extract_pocket(
    protein_pdb,
    ligand_mol2,
    distance_threshold=8.0,
    output_path=None,
    include_heteroatoms=False,
    verbose=False,
):
    """
    Extract the protein pocket residues and heteroatoms within a specified distance of the ligand.

    Parameters:
    - protein_pdb (str): Path to the protein PDB file.
    - ligand_mol2 (str): Path to the ligand MOL2 file.
    - distance_threshold (float): Distance threshold (in Angstroms) for determining the pocket (default: 8.0).
    - output_path (str): Path to save the pocket PDB file (default: None, saves as 'pocket.pdb' in the current directory).
    - include_heteroatoms (bool): Whether to include heteroatoms in the pocket (default: False, includes only water molecules).
    - verbose (bool): Whether to print verbose output for each step (default: False).

    Returns:
    - pred_pocket (PandasPdb): Biopandas PDB object representing the extracted pocket.
    """
    try:
        # Read in protein PDB file
        protein = PandasPdb().read_pdb(protein_pdb)
        if verbose:
            print("[green]Protein PDB file read successfully:[/green]", protein_pdb)
    except IOError:
        print("[red]Error: Could not read protein PDB file:[/red]", protein_pdb)
        return None

    try:
        # Read in ligand MOL2 file
        ligand = PandasMol2().read_mol2(ligand_mol2)
        if verbose:
            print("[green]Ligand MOL2 file read successfully:[/green]", ligand_mol2)
    except IOError:
        print("[red]Error: Could not read ligand MOL2 file:[/red]", ligand_mol2)
        return None

    # Define protein atoms dataframe
    protein_atom = protein.df["ATOM"]
    if verbose:
        print("[grey]Protein atoms dataframe created.[/grey]")
        print("Protein atoms dataframe shape:", protein_atom.shape)

    # Define protein heteroatoms dataframe
    protein_hetatm = protein.df["HETATM"]
    if verbose:
        print("[grey]Protein heteroatoms dataframe created.[/grey]")
        print("Protein heteroatoms dataframe shape:", protein_hetatm.shape)

    # Define ligand non-H atoms dataframe
    ligand_nonh = ligand.df[ligand.df["atom_type"] != "H"]
    if verbose:
        print("[grey]Ligand non-H atoms dataframe created.[/grey]")
        print("Ligand non-H atoms dataframe shape:", ligand_nonh.shape)

    # Create k-d trees for protein atoms and heteroatoms
    protein_atom_tree = cKDTree(protein_atom[["x_coord", "y_coord", "z_coord"]].values)
    protein_hetatm_tree = cKDTree(
        protein_hetatm[["x_coord", "y_coord", "z_coord"]].values
    )
    if verbose:
        print("[grey]K-d trees created for protein atoms and heteroatoms.[/grey]")

    # Find protein atoms within the distance threshold of ligand atoms
    ligand_coords = ligand_nonh[["x", "y", "z"]].values
    nearby_atom_indices = protein_atom_tree.query_ball_point(
        ligand_coords, r=distance_threshold
    )
    if verbose:
        print(
            "[grey]Nearby protein atoms within[/grey]",
            distance_threshold,
            "[grey]Angstroms of ligand atoms found.[/grey]",
        )
        print("Number of nearby protein atoms:", len(nearby_atom_indices))

    # Find protein heteroatoms within the distance threshold of ligand atoms
    nearby_hetatm_indices = protein_hetatm_tree.query_ball_point(
        ligand_coords, r=distance_threshold
    )
    if verbose:
        print(
            "[grey]Nearby protein heteroatoms within[/grey]",
            distance_threshold,
            "[grey]Angstroms of ligand atoms found.[/grey]",
        )
        print("Number of nearby protein heteroatoms:", len(nearby_hetatm_indices))

    # Get unique residues from nearby protein atoms
    pocket_residues = (
        protein_atom.iloc[np.unique(np.concatenate(nearby_atom_indices))][
            ["chain_id", "residue_number", "insertion"]
        ]
        .apply(lambda x: "_".join(x.astype(str)), axis=1)
        .unique()
    )
    if verbose:
        print("[grey]Unique pocket residues extracted.[/grey]")
        print("Number of unique pocket residues:", len(pocket_residues))

    # Get unique heteroatoms from nearby protein heteroatoms
    pocket_heteroatoms = protein_hetatm.iloc[
        np.unique(np.concatenate(nearby_hetatm_indices))
    ]["residue_number"].unique()
    if verbose:
        print("[grey]Unique pocket heteroatoms extracted.[/grey]")
        print("Number of unique pocket heteroatoms:", len(pocket_heteroatoms))

    # Filter protein atoms by pocket residues
    pocket_atoms = protein_atom[
        protein_atom.apply(
            lambda x: "_".join(
                [str(x["chain_id"]), str(x["residue_number"]), str(x["insertion"])]
            ),
            axis=1,
        ).isin(pocket_residues)
    ]
    if verbose:
        print("[grey]Protein atoms filtered by pocket residues.[/grey]")
        print("Number of pocket atoms:", pocket_atoms.shape[0])

    # Filter protein heteroatoms by pocket heteroatoms
    if include_heteroatoms:
        pocket_hetatms = protein_hetatm[
            protein_hetatm["residue_number"].isin(pocket_heteroatoms)
        ]
        if verbose:
            print("[grey]All heteroatoms included in the pocket.[/grey]")
            print("Number of pocket heteroatoms:", pocket_hetatms.shape[0])
    else:
        pocket_hetatms = protein_hetatm[
            (protein_hetatm["residue_number"].isin(pocket_heteroatoms))
            & (protein_hetatm["residue_name"] == "HOH")
        ]
        if verbose:
            print("[grey]Only water molecules included in the pocket.[/grey]")
            print("Number of pocket water molecules:", pocket_hetatms.shape[0])

    # Initialize biopandas object to write out pocket PDB file
    pred_pocket = PandasPdb()
    if verbose:
        print("[grey]Biopandas PDB object initialized.[/grey]")

    # Define the atoms and heteroatoms of the object
    pred_pocket.df["ATOM"], pred_pocket.df["HETATM"] = pocket_atoms, pocket_hetatms
    if verbose:
        print("[grey]Pocket atoms and heteroatoms assigned to the PDB object.[/grey]")

    # Save the created object to a PDB file
    if output_path is None:
        output_path = "pocket.pdb"
    else:
        output_path = os.path.join(output_path, "pocket.pdb")

    try:
        pred_pocket.to_pdb(output_path)
        print("[green]Pocket PDB file saved as:[/green]", output_path)
    except IOError:
        print("[red]Error: Could not save pocket PDB file:[/red]", output_path)

    return pred_pocket

In [10]:
# Extract the pocket for the sample protein-ligand pair
extract_pocket(
    protein_pdb=os.path.join("sample_data", sample_protein_pdb),
    ligand_mol2=os.path.join("sample_data", sample_ligand_mol2),
    distance_threshold=8.0,
    output_path=sample_data_path,
    include_heteroatoms=False,
    verbose=True,
)

Protein PDB file read successfully: sample_data/1dhj_protein.pdb

Ligand MOL2 file read successfully: sample_data/1dhj_ligand.mol2

Protein atoms dataframe created.

Protein atoms dataframe shape:
(4889, 21)

Protein heteroatoms dataframe created.

...

Only water molecules included in the pocket.

Number of pocket water molecules: 32

Biopandas PDB object initialized.

Pocket atoms and heteroatoms assigned to the PDB object.

Pocket PDB file saved as: sample_data/pocket.pdb

In [11]:
%pip install requests

Note: you may need to restart the kernel to use updated packages.


In [12]:
import os

# import requests module
import requests
from pymol import cmd

In [13]:
# initialize a pymol state by first deleting everything
cmd.delete("all")

# load in the created pocket pdb file
cmd.load("sample_data/pocket.pdb")

# add hydrogens to the water molecules
cmd.h_add("sol")

# save the state as a mol2 file
cmd.save("sample_data/pocket.mol2")

In [14]:
def add_mol2_charges(pocket_mol2):
    # upload the pocket mol2 file to the ACC2 API
    r = requests.post(
        "http://78.128.250.156/send_files",
        files={"file[]": open(pocket_mol2, "rb")},
    )

    # obtain ID number for uploaded file
    r_id = list(r.json()["structure_ids"].values())[0]

    # calculate charges using eqeq method
    r_out = requests.get(
        "http://78.128.250.156/calculate_charges?structure_id="
        + r_id
        + "&method=eqeq&generate_mol2=true"
    )

    # save output mol2 file
    open("/content/charged_pocket.mol2", "wb").write(r_out.content)

In [15]:
add_mol2_charges("sample_data/pocket.mol2")

ConnectionError: HTTPConnectionPool(host='78.128.250.156', port=80): Max retries exceeded with url: /send_files (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x1431555a0>: Failed to establish a new connection: [Errno 61] Connection refused'))

The **init** method initializes the Featurizer object with the following parameters:

- atom_codes: A dictionary that maps atomic numbers to specific atom classes.
- atom_labels: A list of labels for the atom classes.
  named_properties: A list of OpenBabel's pybel.Atom properties to include as features.
- save_molecule_codes: A boolean indicating whether to save molecule codes as a feature.
- custom_properties: A list of custom callable functions to compute additional features.
- smarts_properties: A list of SMARTS patterns to match and include as features.
- smarts_labels: A list of labels for the SMARTS patterns.

The compile_smarts method compiles the SMARTS patterns for efficient matching.
The encode_num method performs one-hot encoding of the atomic number based on the defined atom classes.
The find_smarts method finds substructures in the molecule that match the SMARTS patterns and returns a binary feature matrix.
The get_features method extracts the features from a given molecule. It ignores hydrogens and dummy atoms, and combines the one-hot encoded atomic numbers, named properties, custom properties, and SMARTS features into a feature matrix.
The to_pickle and from_pickle methods allow saving and loading the Featurizer object to/from a pickle file.


In [16]:
class Featurizer:
    def __init__(
        self,
        atom_codes=None,
        atom_labels=None,
        named_properties=None,
        save_molecule_codes=True,
        custom_properties=None,
        smarts_properties=None,
        smarts_labels=None,
    ):
        # initialize list to store names of all features in the correct order
        self.FEATURE_NAMES = []

        # validate and process atom codes and labels
        if atom_codes is not None:
            if not isinstance(atom_codes, dict):
                raise TypeError(
                    "Atom codes should be dict, got %s instead" % type(atom_codes)
                )

            codes = set(atom_codes.values())
            for i in range(len(codes)):
                if i not in codes:
                    raise ValueError("Incorrect atom code %s" % i)

            self.NUM_ATOM_CLASSES = len(codes)
            self.ATOM_CODES = atom_codes

            if atom_labels is not None:
                if len(atom_labels) != self.NUM_ATOM_CLASSES:
                    raise ValueError(
                        "Incorrect number of atom labels: "
                        "%s instead of %s" % (len(atom_labels), self.NUM_ATOM_CLASSES)
                    )

            else:
                atom_labels = ["atom%s" % i for i in range(self.NUM_ATOM_CLASSES)]

            self.FEATURE_NAMES += atom_labels

        else:
            self.ATOM_CODES = {}
            metals = (
                [3, 4, 11, 12, 13]
                + list(range(19, 32))
                + list(range(37, 51))
                + list(range(55, 84))
                + list(range(87, 104))
            )

            # List of tuples (atomic_num, class_name) with atom types to encode
            atom_classes = [
                (5, "B"),
                (6, "C"),
                (7, "N"),
                (8, "O"),
                (15, "P"),
                (16, "S"),
                (34, "Se"),
                ([9, 17, 35, 53], "halogen"),
                (metals, "metal"),
            ]

            for code, (atom, name) in enumerate(atom_classes):
                if type(atom) is list:
                    for a in atom:
                        self.ATOM_CODES[a] = code

                else:
                    self.ATOM_CODES[atom] = code

                self.FEATURE_NAMES.append(name)

            self.NUM_ATOM_CLASSES = len(atom_classes)

        # validate and process named properties
        if named_properties is not None:
            if not isinstance(named_properties, (list, tuple, np.ndarray)):
                raise TypeError("named_properties must be a list")

            allowed_props = [
                prop for prop in dir(openbabel.pybel.Atom) if not prop.startswith("__")
            ]

            for prop_id, prop in enumerate(named_properties):
                if prop not in allowed_props:
                    raise ValueError(
                        "named_properties must be in pybel.Atom attributes,"
                        " %s was given at position %s" % (prop_id, prop)
                    )

            self.NAMED_PROPS = named_properties

        else:
            # pybel.Atom properties to save
            self.NAMED_PROPS = [
                "hyb",
                "heavydegree",
                "heterodegree",
                "partialcharge",
            ]

        self.FEATURE_NAMES += self.NAMED_PROPS

        if not isinstance(save_molecule_codes, bool):
            raise TypeError(
                "save_molecule_codes should be bool, got %s "
                "instead" % type(save_molecule_codes)
            )

        self.save_molecule_codes = save_molecule_codes

        if save_molecule_codes:
            # Remember if an atom belongs to the ligand or to the protein
            self.FEATURE_NAMES.append("molcode")

        # process custom callable properties
        self.CALLABLES = []

        if custom_properties is not None:
            for i, func in enumerate(custom_properties):
                if not callable(func):
                    raise TypeError(
                        "custom_properties should be list of"
                        " callables, got %s instead" % type(func)
                    )

                name = getattr(func, "__name__", "")

                if name == "":
                    name = "func%s" % i

                self.CALLABLES.append(func)

                self.FEATURE_NAMES.append(name)

        # process SMARTS properties and labels
        if smarts_properties is None:
            # SMARTS definition for other properties
            self.SMARTS = [
                "[#6+0!$(*~[#7,#8,F]),SH0+0v2,s+0,S^3,Cl+0,Br+0,I+0]",
                "[a]",
                "[!$([#1,#6,F,Cl,Br,I,o,s,nX3,#7v5,#15v5,#16v4,#16v6,*+1,*+2,*+3])]",
                "[!$([#6,H0,-,-2,-3]),$([!H0;#7,#8,#9])]",
                "[r]",
            ]

            smarts_labels = ["hydrophobic", "aromatic", "acceptor", "donor", "ring"]

        elif not isinstance(smarts_properties, (list, tuple, np.ndarray)):
            raise TypeError("smarts_properties must be a list")

        else:
            self.SMARTS = smarts_properties

        if smarts_labels is not None:
            if len(smarts_labels) != len(self.SMARTS):
                raise ValueError(
                    "Incorrect number of SMARTS labels: %s"
                    " instead of %s" % (len(smarts_labels), len(self.SMARTS))
                )

        else:
            smarts_labels = ["smarts%s" % i for i in range(len(self.SMARTS))]

        # Compile SMARTS patterns for matching
        self.compile_smarts()

        self.FEATURE_NAMES += smarts_labels

    # define function to compile SMARTS patterns for efficient matching
    def compile_smarts(self):
        self.__PATTERNS = []

        for smarts in self.SMARTS:
            self.__PATTERNS.append(openbabel.pybel.Smarts(smarts))

    # define function to encode the atomic number using one-hot encoding
    def encode_num(self, atomic_num):
        if not isinstance(atomic_num, int):
            raise TypeError(
                "Atomic number must be int, %s was given" % type(atomic_num)
            )

        encoding = np.zeros(self.NUM_ATOM_CLASSES)

        try:
            encoding[self.ATOM_CODES[atomic_num]] = 1.0

        except:
            pass

        return encoding

    # define function to find substructures in the molecule that match the SMARTS patterns
    def find_smarts(self, molecule):
        if not isinstance(molecule, openbabel.pybel.Molecule):
            raise TypeError(
                "molecule must be pybel.Molecule object, %s was given" % type(molecule)
            )

        features = np.zeros((len(molecule.atoms), len(self.__PATTERNS)))

        for pattern_id, pattern in enumerate(self.__PATTERNS):
            atoms_with_prop = (
                np.array(list(*zip(*pattern.findall(molecule))), dtype=int) - 1
            )

            features[atoms_with_prop, pattern_id] = 1.0

        return features

    # define function to extract the features from the molecule
    def get_features(self, molecule, molcode=None):
        if not isinstance(molecule, openbabel.pybel.Molecule):
            raise TypeError(
                "molecule must be pybel.Molecule object,"
                " %s was given" % type(molecule)
            )

        if molcode is None:
            if self.save_molecule_codes is True:
                raise ValueError(
                    "save_molecule_codes is set to True,"
                    " you must specify code for the molecule"
                )

        elif not isinstance(molcode, (float, int)):
            raise TypeError("motlype must be float, %s was given" % type(molcode))

        coords = []
        features = []
        heavy_atoms = []

        for i, atom in enumerate(molecule):
            # ignore hydrogens and dummy atoms (they have atomicnum set to 0)
            if atom.atomicnum > 1:
                heavy_atoms.append(i)

                coords.append(atom.coords)

                features.append(
                    np.concatenate(
                        (
                            self.encode_num(atom.atomicnum),
                            [atom.__getattribute__(prop) for prop in self.NAMED_PROPS],
                            [func(atom) for func in self.CALLABLES],
                        )
                    )
                )

        coords = np.array(coords, dtype=np.float32)

        features = np.array(features, dtype=np.float32)

        if self.save_molecule_codes:
            features = np.hstack((features, molcode * np.ones((len(features), 1))))

        features = np.hstack([features, self.find_smarts(molecule)[heavy_atoms]])

        if np.isnan(features).any():
            raise RuntimeError("Got NaN when calculating features")

        return coords, features

    # define function to save the Featurizer to a pickle file
    def to_pickle(self, fname="featurizer.pkl"):
        # patterns can't be pickled, we need to temporarily remove them
        patterns = self.__PATTERNS[:]

        del self.__PATTERNS

        try:
            with open(fname, "wb") as f:
                pickle.dump(self, f)

        finally:
            self.__PATTERNS = patterns[:]

    @staticmethod
    def from_pickle(fname):
        with open(fname, "rb") as f:
            featurizer = pickle.load(f)

        featurizer.compile_smarts()

        return featurizer

In [17]:
!pwd

/Users/marvinprakash/Codes/PLB/HAC-net


In [18]:
elements_xml = "HAC-Net-Repo/HACNet/element_features.xml"

In this improved version of the code:

- The variable names have been updated to be more meaningful and descriptive.
- The leading double underscores have been removed from function names.
- Docstrings have been added to the main function and the inner functions to describe their purpose, input parameters, and return values.
- Context managers (with statements) have been used for file handling with OpenBabel's `pybel.readfile`.
- Intermediate variable names have been improved for better readability.
- A constant `MINIMUM_ATOMIC_NUMBER` has been defined to replace the hardcoded value.
- The returned variables have been given more descriptive names: atom_features and atom_vdw_radii.


In [19]:
import xml.etree.ElementTree as ET

MINIMUM_ATOMIC_NUMBER = 2


def prepare_data(pocket_mol2_charged, ligand_mol2, elements_xml):
    # define function to extract features from the binding pocket mol2 file
    def __get_pocket():
        # define pocket from input pocket_mol2 file
        pocket = next(openbabel.pybel.readfile("mol2", pocket_mol2_charged))

        # obtain pocket coordinates and features
        pocket_coords, pocket_features = featurizer.get_features(pocket, molcode=-1)

        # obtain Van der Waals radii of pocket atoms
        pocket_vdw = parse_mol_vdw(mol=pocket, element_dict=element_dict)

        yield (pocket_coords, pocket_features, pocket_vdw)

    # define function to extract information from elements_xml file
    def parse_element_description(xml_file):
        # initialize dictionary to store element chemical information
        element_info_dict = {}

        # parse and define elements_xml file
        element_info_xml = ET.parse(xml_file)

        # for each element in file
        for element in element_info_xml.iter():
            # if 'comment' is keys of element attributes
            if "comment" in element.attrib.keys():
                # continue
                continue

            # if 'comment' not in keys of element attributes
            else:
                # save the attribute to the dictionary value to 'number
                element_info_dict[int(element.attrib["number"])] = element.attrib

        # return dictionary containing element information
        return element_info_dict

    # define function to create a list of van der Waals radii for a molecule
    def parse_mol_vdw(mol, element_dict):
        # initialize list to store Van der Waals radii
        vdw_list = []

        # for each atom in molecule
        for atom in mol.atoms:
            # if the atom is not Hydrogen
            if int(atom.atomicnum) >= 2:
                # append the Van der Waals radius to the list
                vdw_list.append(float(element_dict[atom.atomicnum]["vdWRadius"]))

        # return the Van der Waals list as a numpy array
        return np.asarray(vdw_list)

    # read in elements_xml and store important information in dictionary
    element_dict = parse_element_description(elements_xml)

    # define featurizer object
    featurizer = Featurizer()

    # define object for getting features of pocket
    pocket_generator = __get_pocket()

    # read ligand file using pybel
    ligand = next(openbabel.pybel.readfile("mol2", ligand_mol2))

    # extract coordinates, 19 features, and Van der Waals radii from pocket atoms
    pocket_coords, pocket_features, pocket_vdw = next(pocket_generator)

    # extract coordinates, and 19 features from ligand atoms
    ligand_coords, ligand_features = featurizer.get_features(ligand, molcode=1)

    # extract Van der Waals radii from ligand atoms
    ligand_vdw = parse_mol_vdw(mol=ligand, element_dict=element_dict)

    # define centroid to be the center of the ligand
    centroid = ligand_coords.mean(axis=0)

    # normalize ligand coordinates with respect to centroid
    ligand_coords -= centroid

    # normalize pocket coordinates with respect to centroid
    pocket_coords -= centroid

    # assemble the features into one large numpy array where rows are heavy atoms, and columns are coordinates and features
    data = np.concatenate(
        (
            np.concatenate((ligand_coords, pocket_coords)),
            np.concatenate((ligand_features, pocket_features)),
        ),
        axis=1,
    )

    # concatenate van der Waals radii into one numpy array
    vdw_radii = np.concatenate((ligand_vdw, pocket_vdw))

    # return properly formatted coordinates, features, and Van der Waals radii
    return data, vdw_radii

In [20]:
pocket_mol2_charged = "sample_data/pocket_charged.mol2"
ligand_mol2 = "sample_data/1dhj_ligand.mol2"


prep_data, prep_vdw = prepare_data(pocket_mol2_charged, ligand_mol2, elements_xml)

In [21]:
def voxelize_atoms(xyz_array, features, voxel_dimensions):
    """
    Voxelize the input data by assigning each atom to a voxel based on its coordinates
    and adding the atom's features to the corresponding voxel.

    Args:
        xyz_array (numpy.ndarray): Array containing the coordinates of the atoms.
        features (numpy.ndarray): Array containing the features associated with each atom.
        voxel_dimensions (tuple): Dimensions of the voxel grid (num_channels, x_dim, y_dim, z_dim).

    Returns:
        numpy.ndarray: Voxelized data with shape (num_channels, x_dim, y_dim, z_dim).
    """
    num_channels, x_dim, y_dim, z_dim = voxel_dimensions
    voxel_data = np.zeros((num_channels, x_dim, y_dim, z_dim), dtype=np.float32)

    x_min, x_max = np.min(xyz_array[:, 0]), np.max(xyz_array[:, 0])
    y_min, y_max = np.min(xyz_array[:, 1]), np.max(xyz_array[:, 1])
    z_min, z_max = np.min(xyz_array[:, 2]), np.max(xyz_array[:, 2])

    x_center = (x_min + x_max) / 2
    y_center = (y_min + y_max) / 2
    z_center = (z_min + z_max) / 2

    voxel_size = 48
    x_min, x_max = x_center - voxel_size / 2, x_center + voxel_size / 2
    y_min, y_max = y_center - voxel_size / 2, y_center + voxel_size / 2
    z_min, z_max = z_center - voxel_size / 2, z_center + voxel_size / 2

    for atom_index in range(xyz_array.shape[0]):
        x, y, z = xyz_array[atom_index]

        if not (x_min <= x < x_max and y_min <= y < y_max and z_min <= z < z_max):
            continue

        voxel_x = int((x - x_min) / (x_max - x_min) * x_dim)
        voxel_y = int((y - y_min) / (y_max - y_min) * y_dim)
        voxel_z = int((z - z_min) / (z_max - z_min) * z_dim)

        voxel_data[:, voxel_x, voxel_y, voxel_z] += features[atom_index]

    return voxel_data

In [22]:
import torch

In [23]:
# prepare voxelized data
prep_data_vox = voxelize_atoms(
    prep_data[:, 0:3], prep_data[:, 3:], [prep_data.shape[1] - 3, 48, 48, 48]
)

# add an additional axis and convert to a tensor
prep_data_vox = torch.tensor(prep_data_vox[np.newaxis, ...])

In [26]:
prep_data_vox.shape

# Save the voxelized numpy array to a file
np.save("sample_data/voxelized_data.npy", prep_data_vox.numpy())

In [25]:
prep_data.shape

(506, 22)

In [ ]:
# Store this vox_data in a file

In [20]:
# Extract the pocket for the sample protein-ligand pair
extract_pocket(
    protein_pdb=os.path.join("sample_data", sample_protein_pdb),
    ligand_mol2=os.path.join("sample_data", sample_ligand_mol2),
    distance_threshold=8.0,
    output_path=sample_data_path,
    include_heteroatoms=False,
    verbose=True,
)

# initialize a pymol state by first deleting everything
cmd.delete("all")

# load in the created pocket pdb file
cmd.load("sample_data/pocket.pdb")

# add hydrogens to the water molecules
cmd.h_add("sol")

# save the state as a mol2 file
cmd.save("sample_data/pocket.mol2")


# Add Charges


# Voxelize
pocket_mol2_charged = "sample_data/pocket_charged.mol2"
ligand_mol2 = "sample_data/1dhj_ligand.mol2"


prep_data, prep_vdw = prepare_data(pocket_mol2_charged, ligand_mol2, elements_xml)

Protein PDB file read successfully: sample_data/1dhj_protein.pdb

Ligand MOL2 file read successfully: sample_data/1dhj_ligand.mol2

Protein atoms dataframe created.

Protein atoms dataframe shape:
(4889, 21)

Protein heteroatoms dataframe created.

...

Number of pocket water molecules: 32

Biopandas PDB object initialized.

Pocket atoms and heteroatoms assigned to the PDB object.

Pocket PDB file saved as: sample_data/pocket.pdb

NameError: name 'cmd' is not defined